In [7]:
import requests

# get keys from file

In [3]:
with open("keys1.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()
    

# Useful functions


In [4]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

link to user info call: https://developers.tiktok.com/doc/research-api-specs-query-user-info?enter_method=left_navigation


In [5]:
def get_user_info(username, access_token,fields ="display_name, bio_description, avatar_url, is_verified, follower_count, following_count, likes_count, video_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/info/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

link to video call: https://developers.tiktok.com/doc/research-api-specs-query-videos?enter_method=left_navigation

In [16]:
def query_video_per_username(access_token,username):
    
    endpoint = "https://open.tiktokapis.com/v2/research/video/query/"
    query_params = {
    "fields": "id,video_description,create_time,region_code,share_count,view_count,like_count,comment_count,music_id,hashtag_names,username,effect_ids,playlist_id,voice_to_text",
    }
    query = {
        "and": [ {"operation": "IN", "field_name": "username", "field_values": [username]} ]
    }
    query_body = {
    "query": query,
    "start_date": '20250120',
    "end_date": '20250130', #max period between start and end date is 30 days
    "max_count": 3 #max here is 100 per call
    }
    
    # Set headers
    #access token expires in 2 hours
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
        # Make the request
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers, timeout=30)
    
   
    if response.status_code == 200:
        
        data = response.json().get("data", {})
        videos = data.get("videos", [])

        return (videos)


    else:
        
        return ([response])
    

# Try it out!

In [8]:
access_token = get_access_token(client_key, client_secret)

In [23]:
get_user_info('ecofran', access_token)

{'likes_count': 14667073,
 'video_count': 554,
 'avatar_url': 'https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/7336807972263165995~c5_168x168.jpeg?lk3s=a5d48078&nonce=44576&refresh_token=0462af94c17a14079e81afe77bdd1d9a&x-expires=1738857600&x-signature=3l%2BVnEp%2FZ1ulWj9Fgl6hNJziow4%3D&shp=a5d48078&shcp=8aecc5ac',
 'bio_description': 'I turn glass into sand for coastal restoration\nIG: franzeeska YT: ecofran_',
 'display_name': 'FRANZISKA ♻️',
 'follower_count': 313667,
 'following_count': 357,
 'is_verified': False}

In [24]:
query_video_per_username(access_token,'ecofran')

[{'music_id': 6705083959446865921,
  'share_count': 2,
  'id': 7463532349708569902,
  'like_count': 193,
  'hashtag_names': [],
  'region_code': 'US',
  'username': 'ecofran',
  'video_description': 'I never thought I’d see our islands and glass recycling facility covered in snow. It’s quite magical ❄️',
  'view_count': 1821,
  'comment_count': 12,
  'create_time': 1737739113},
 {'comment_count': 30,
  'like_count': 656,
  'share_count': 2,
  'username': 'ecofran',
  'video_description': 'I never thought I would see snow covering the islands ❄️❄️❄️ I’m taking in the magic of it all while grappling with our changing climate and what it means for Louisiana 💙',
  'create_time': 1737665814,
  'hashtag_names': [],
  'id': 7463217787041795371,
  'music_id': 7463217763738929963,
  'region_code': 'US',
  'view_count': 2972,
  'voice_to_text': "Louisiana snow update. I'm at my islands and they're covered in snow. It's pretty magical looking. Um, one of the best parts is that it looks like all t